In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt 
import missingno as msno
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
import plotly.express as px
import scipy
from scipy import stats
from scipy.stats import pearsonr
import matplotlib

In [ ]:
# Versions
print("Les version des librairies : ")
print("Numpy        : " + np.version.full_version)
print("Pandas       : " + pd.__version__)
print("Seaborn      : " + sns.__version__)
print("missingno    : " + msno.__version__)
print("matplotlib : " + matplotlib.__version__)

### Les Fonctions

In [ ]:
# function pour afficher les occurrences par indicateurs
def occurrenceVisualization(dataframe):
    dfCount = dataframe.value_counts().reset_index().values.tolist() 
    dfCount_colonne = np.array(dfCount)
    print(f"Dataframe : {dataframe.name}")
    print(tabulate(dfCount_colonne, headers=["Valeur unique", "Nombre d'occurrences"], tablefmt="grid"))

# fonction pour visualiser les caractéristiques du fichier
def caracteristiquesFichier(fichier):
    DataFrame = pd.read_csv(fichier)     # Lecture du fichier csv
    DataFrame.describe()
    display(DataFrame)
    print(DataFrame.shape)
    print(DataFrame.dtypes)
    print(f'La moyenne de valeurs manquantes est de : {DataFrame.isna().mean()}')
    return DataFrame

# fonction pour visualiser le taux de remplissage
def PlotTauxDeRemplissage(dataframe, figureName, colorOfBar="seagreen",sort=None):
    plt.figure()
    plt.title('Taux de remplissage par indicateur',fontsize=15) #titre
    msno.bar(dataframe,fontsize=12, color=colorOfBar,sort=sort) # taux de remplissage
    plt.tight_layout()      # pour que rien ne sorte du format de l'image
    plt.savefig(figureName) #sauvegarde de la figure
    plt.show()



# fonction pour calculer le nombre de doublons et les supprimer
def areThereDuplicates(dataframe, subset=['Country Code', 'Short Name']):

    nombre_doublons_sc = dataframe.duplicated(subset=subset).sum()
    print("Nombre de doublons dans le DataFrame :", nombre_doublons_sc)

    if nombre_doublons_sc>0:
        dataframe.drop_duplicates(subset=subset, inplace=True)
        print("--- suppression des doublons --- ")  

    return dataframe


# Fonction pour supprimer les colonnes vide
# seuil entre 0 et 1
def cleanFile(dataframe, seuil):  

    # Calcul du nombre moyen de NaN 
    nbEmptyCol = np.array(np.where(dataframe.isna().mean()>=seuil)).shape[1]
    

    if nbEmptyCol == 1:
        print(f"Il y a {nbEmptyCol} colonne vide, qui est à l'indice {np.where(dataframe.isna().mean())}")
        dataframe.dropna(axis=1, how='all', inplace=True) #suppression des colonnes entièrement vide
        print(f"--- suppression de la colonne vide à plus de {seuil:0.1%} --- ")
    
    elif nbEmptyCol==0:
        print(f"Il n'y a pas colonnes vides à plus de {seuil:0.1%}")
    else:
        print(f"Il y a {nbEmptyCol} colonnes vides, qui sont aux indices {np.where(dataframe.isna().mean())}")
        dataframe.dropna(axis=1, how='all', inplace=True) #suppression des colonnes entièrement vide
        print(f"--- suppression des {nbEmptyCol} colonnes vides à plus de {seuil:0.1%}--- ")


    return dataframe
    

### Fichier 1 : CountrySeries

In [ ]:
# Fichier 1 : CountrySeries

# Lecture et préparation du fichier
CountrySeries = caracteristiquesFichier("EdStatsCountry-Series.csv")

# Nettoyage des données
cleanedDf     = cleanFile(CountrySeries, 0.9)   #colonnes vides
cleanerDf     = areThereDuplicates(cleanedDf,subset=['CountryCode', 'SeriesCode']) #doublons

# Nombre de pays par seriesCode
sommescs = cleanerDf.groupby('SeriesCode')['CountryCode'].count()

# Nb de pays
paysscs = cleanerDf["CountryCode"].nunique()


# ------- #
# FIGURE 1 : Nombre de pays par code de série
# ------- #
plt.bar(height= sommescs, x=sommescs.index)
plt.xlabel('SeriesCode')
plt.ylabel('Nombre de pays')
plt.title('Nombre de pays par code de série')
plt.xticks(rotation=90)  # Rotation des étiquettes de l'axe x pour une meilleure lisibilité
plt.show()
# ------- #



# -------------------------------- #
# Résumé  Stat
print(f"En moyenne, les codes de séries sont associées à {sommescs.mean():0.1f} ± {sommescs.std():0.1f} pays.")
print(f"Deux codes de série (SP.POP.GROW; SP.POP.TOTL) sont associés à tous les pays (i.e., {paysscs}), ce qui augmente la moyenne.")
print(f"50% (i.e. la médiane) des codes de séries sont associées à {sommescs.median():0.1f} pays.")

# ------- #
# FIGURE 2 : Distribution statistique du nombre de pays associée aux codes de séries
# ------- #
plt.figure()
plt.boxplot(sommescs)
plt.ylim((0,25))
plt.ylabel('Nombre de pays')
plt.title('Distribution statistique du nombre de pays associée aux codes de séries')

plt.text(1.05, 19, "Maximum") # 1.5 fois l'espace interquartile
plt.text(0.65, 14, "3eme quartile (75%)") # 75% des series code sont inférieurs à cette valeur
plt.text(1.1, 13, "Median (50%)", color='orange') # 50% des series code sont inférieurs à cette valeur
plt.text(0.65, 4, "1er quartile (25%)")  # 25% des series code sont inférieurs à cette valeur
plt.text(1.1, 3.8, "Minimum") # 1.5 fois l'espace interquartile
plt.show()
# ------- #



# ------- #
# FIGURE 3 : Distribution des series codes
# ------- #
sns.displot(sommescs, kde=True, bins=20)
plt.ylabel('Nombre de codes de séries')
plt.show()
# ------- #


StatsCountry-Series a 3 colonnes et 613 lignes (aprés supression de la dernière colonne sans donnée)
Les trois colonnes sont de types Object 
On trouve des pays, des variables et leurs descriptions
On y trouve 211 pays différents

### Fichier 2 : Country

In [ ]:
# Fichier 2 : Country

# Lecture et préparation du fichier
Country = caracteristiquesFichier("EdStatsCountry.csv")

# Nettoyage des données
cleanedDf     = cleanFile(Country, 0.8)  
cleanerDf     = areThereDuplicates(cleanedDf,subset=['Country Code', 'Short Name'])
PlotTauxDeRemplissage(cleanerDf, "test.png")

# Nb de pays
payssc = cleanerDf["Country Code"].nunique()
print('Le nombre de pays dans Country est de ', payssc)

# Afficher le nb d'occurences
occurrenceVisualization(cleanerDf["IMF data dissemination standard"])
occurrenceVisualization(cleanerDf["Government Accounting concept"])
occurrenceVisualization(cleanerDf["SNA price valuation"])
occurrenceVisualization(cleanerDf["Lending category"])
occurrenceVisualization(cleanerDf["System of National Accounts"])
occurrenceVisualization(cleanerDf["Alternative conversion factor"])
occurrenceVisualization(cleanerDf["PPP survey year"])
occurrenceVisualization(cleanerDf["Balance of Payments Manual in use"])
occurrenceVisualization(cleanerDf["External debt Reporting status"])

# plot Région
plt.figure()
plt.barh(y = cleanerDf.groupby('Region')[['Short Name']].count().reset_index().sort_values(by='Short Name')['Region'], width = cleanerDf.groupby('Region')[['Short Name']].count()['Short Name'].sort_values())
plt.title('Nombre de pays par Région')
# Afficher le résultat à coté
df = (Country.groupby("Region")[["Short Name"]]
    .count()["Short Name"]
    .sort_values(ascending=True))
val = df.tolist()
plt.text(val[6] - 5.5,  5.8, val[6], color="white", fontweight="bold")
plt.text(val[5] - 5.5,  4.8, val[5], color="white", fontweight="bold")
plt.text(val[4] - 5.5,  3.8, val[4], color="white", fontweight="bold")
plt.text(val[3] - 5.5,  2.8, val[3], color="white", fontweight="bold")
plt.text(val[2] - 5.5,  1.8, val[2], color="white", fontweight="bold")
plt.text(val[1] - 3,  0.8, val[1], color="white", fontweight="bold")
plt.text(val[0] - 2.5, -0.2, val[0], color="white", fontweight="bold")
plt.show()


# Grouper les données par groupe de revenu et compter le nombre de pays dans chaque groupe
IncomeGroupPays = Country.groupby("Income Group")["Short Name"].count().reset_index().sort_values(by="Short Name", ascending=False)

plt.figure(figsize=(12, 6))
plt.barh(y=IncomeGroupPays["Income Group"], width=IncomeGroupPays["Short Name"], color='skyblue')
plt.title("Nombre de pays par groupe de revenu", fontname="Arial", fontsize=18, fontweight="bold")
plt.xlabel("Nombre de pays")
plt.ylabel("Groupe de revenu")
# Ajouter les annotations du nombre de pays à côté de chaque barre
for i, (income_group, count) in enumerate(zip(IncomeGroupPays["Income Group"], IncomeGroupPays["Short Name"])):
    plt.text(count + 2, i, str(count), color="black", fontweight="bold", fontsize=12, va='center')
plt.tight_layout()
plt.show()



# Regroupement des pays par groupe de revenus
sns.countplot(y="Income Group",hue="Region",data=Country,orient="h",order=["High income: OECD","High income: nonOECD","Upper middle income","Lower middle income","Low income",],palette=sns.color_palette('bright'),)
plt.xlabel("Nombre de pays", fontsize=12)
plt.ylabel("Groupe de revenus", fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc=0, borderaxespad=0.0, fontsize=12)
plt.title("Nombre de pays par groupe de revenus par région", fontsize=14, fontweight="bold")
plt.grid(False)
plt.show()

Dans StatsCountry se trouve 241 lignes et 31 colonnes (aprés suppresion de la dernière colonne sans donnée)
Aucun doublon se trouve dans le DataFrame

### Fichier 3 : Data

In [ ]:
# Fichier 3 : Data
# Lecture et préparation du fichier
Data = caracteristiquesFichier("EdStatsData.csv")

# Nettoyage des données
cleanedDf     = cleanFile(Data,1)  
cleanerDf     = areThereDuplicates(cleanedDf,subset=['Country Code','Indicator Name'])

# Nb de pays
payssd = cleanerDf["Country Code"].nunique()
print('Le nombre de pays dans Data est de ', payssd)

# Nb d'indicateurs unique 
indicatorsd = Data['Indicator Code'].nunique()
print ('Le nombre d indicateurs dans Data est de ', indicatorsd)

#taux de remplissage
TauxRemplissage = Data.isna().mean()
TauxRemplissageEff = TauxRemplissage.iloc[4:] # Le tx de remplissage de 1970 à 2100
TauxDeVide = (1 - TauxRemplissageEff)*100 
temps=TauxRemplissage.iloc[4:].index
plt.figure()
ax=plt.gca()
plt.plot(temps, TauxDeVide,'.-')

plt.ylabel('Taux de remplissage (%)')
plt.xlabel('Temps (Année)')
plt.title('Taux de remplissage en fonction du Temps')
ax.xaxis.set_major_locator(MultipleLocator(10))
ax.xaxis.set_minor_locator(MultipleLocator(1))


In [ ]:
Data.isna().mean().mean()

### Fichier 4 : FootNote

In [ ]:
# Fichier 4 : FootNote
# Lecture et préparation du fichier
FootNote = caracteristiquesFichier("EdStatsFootNote.csv")

# Nettoyage des données
cleanedDf     = cleanFile(FootNote,0.8)  
cleanerDf     = areThereDuplicates(cleanedDf,subset=['CountryCode','SeriesCode', 'Year'])

# Nb de pays
paysfn = cleanerDf["CountryCode"].nunique()
print('Le nombre de pays dans FootNote est de ', paysfn)

# Nb d'indicateurs unique 
indicatorfn = FootNote['SeriesCode'].nunique()
print ('Le nombre d indicateurs dans FootNote est de ', indicatorfn)

# FIGURE : Distribution des series codes
# ------- #
# Nombre de pays par seriesCode
sommesfn = cleanerDf.groupby('SeriesCode')['CountryCode'].count()
sns.displot(sommesfn, kde=True, bins=20)
plt.ylabel('Nombre de codes de séries')


### Fichier 5 : Series

In [ ]:
# Fichier 5 : Series
# Lecture et préparation du fichier
Series = caracteristiquesFichier("EdStatsSeries.csv")

# Nettoyage des données

cleanedDf     = cleanFile(Series,0.8)  
cleanerDf     = areThereDuplicates(cleanedDf,subset=['Series Code'])

# Nb d'indicateurs unique 
indicatorSeries = Series['Series Code'].nunique()
print ('Le nombre d indicateurs dans Series est de ', indicatorSeries)

#taux de remplissage
TauxRemplissage = Series.isna().mean()
TauxRemplissageEff = TauxRemplissage.iloc[5:] # Le tx de remplissage à oartir de Unit of measure
TauxDeVide = (1 - TauxRemplissageEff)*100 
indicateurs=TauxRemplissage.iloc[5:].index
plt.figure(figsize=(10,15))
ax=plt.gca()
plt.plot(TauxDeVide, indicateurs,'.-')

plt.ylabel('Indicateurs')
plt.xlabel('Taux de remplissage (%)')
plt.title('Taux de remplissage des indicateurs')


### Liste des pays

In [ ]:
# Liste de pays par fichier
list_pays1 = list(CountrySeries['CountryCode'])
list_pays2 = list(Country['Country Code'])
list_pays3 = list(Data['Country Code'])
list_pays4 = list(FootNote['CountryCode'])

# concaténation des listes pour n'en faire qu'une seule
list_paysTot = [list_pays1, list_pays2, list_pays3, list_pays4]

# Suppression des doublons
liste_PaysTotal = []
for liste in [list_pays1, list_pays2, list_pays3, list_pays4]:
    for element in liste:
        if element not in liste_PaysTotal:
            liste_PaysTotal.append(element)
liste_PaysTotal = pd.DataFrame(liste_PaysTotal, columns=['Country Code'])
liste_PaysTotal




In [ ]:
# Les régions qui ne sont pas comprises dans la liste des pays sont :
list_regions = Country[-Country['Country Code'].isin(CountrySeries['CountryCode'])][['Short Name']].sort_values(by = 'Short Name')
list_regions2 = Country[-Country['Country Code'].isin(FootNote['CountryCode'])][['Short Name']].sort_values(by = 'Short Name')
list_regions3 = Data[-Data['Country Code'].isin(Country['Country Code'])][['Country Name']]
list_regions3.drop_duplicates(inplace=True)
list_regions4 = Data[-Data['Country Code'].isin(CountrySeries['CountryCode'])][['Country Name']].sort_values(by = 'Country Name')
list_regions3 = list_regions3.rename(columns={'Country Name': 'Short Name'})
list_regions4 = list_regions4.rename(columns={'Country Name': 'Short Name'})

# concaténation des listes pour n'en faire qu'une seule
list_RegionsTot = pd.concat([list_regions, list_regions2, list_regions3, list_regions4], ignore_index=True)
list_RegionsTot

# Suppression des doublons
list_RegionsTotal = list_RegionsTot.drop_duplicates()
list_RegionsTotal

# ajout des country code associés


list_RegionsTotal['Country Code'] = np.nan
for i,name in enumerate(list_RegionsTotal['Short Name']) :
    
    if name in np.array(Country['Short Name']): 
        index_name=np.where(Country['Short Name']==name)
        list_RegionsTotal.iloc[i,1]= Country['Country Code'][index_name[0][0]]

    elif name in np.array(Data['Country Name']): 
        index_name=np.where(Data['Country Name']==name)
        list_RegionsTotal.iloc[i,1]= Data['Country Code'][index_name[0][0]]


In [ ]:
list_RegionsTotal.shape

In [ ]:
# supression des country code des régions qui ne sont pas des pays.
liste_PaysFinal = liste_PaysTotal[-liste_PaysTotal['Country Code'].isin(list_RegionsTotal['Country Code'])][['Country Code']].sort_values(by = 'Country Code')
liste_PaysFinal

liste_PaysFinal['Short Name'] = np.nan
for i,code in enumerate(liste_PaysFinal['Country Code']) :
    
    if code in np.array(Country['Country Code']): 
        index_name=np.where(Country['Country Code']==code)
        liste_PaysFinal.iloc[i,1]= Country['Short Name'][index_name[0][0]]

    elif code in np.array(Data['Country Code']): 
        index_name=np.where(Data['Country Code']==code)
        liste_PaysFinal.iloc[i,1]= Data['Country Name'][index_name[0][0]]

liste_PaysFinal

# Probleme nombre de lignes

### Sélection des indicateurs 

Après avoir parcouru le site de la banque mondiale, j'ai selectionné 9 indicateurs: 

EDUCATION :

1 - Total enrollment in tertiary education (ISCED 5 to 8), regardless of age, expressed as a percentage of the total population of the five-year age group following on from secondary school leaving. (Etude supérieur)

2 -Total enrollment in upper secondary education, regardless of age, expressed as a percentage of the total population of official upper secondary education age. (Lycée)

TECHNIQUE : 

3 - Internet users are individuals who have used the Internet (from any location) in the last 3 months. The Internet can be used via a computer, mobile phone, personal digital assistant, games machine, digital TV etc.

4 - Personal computers are self-contained computers designed to be used by a single individual.

DEMOGRAPHIQUE :

5 - Total, population

6 - Population in thousands, age 15-19, total is the total population of 15-19 year olds in thousands estimated by Barro-Lee.

7 - Population in thousands, age 20-24, total is the total population of 20-24 year olds in thousands estimated by Barro-Lee.

8 - Population in thousands, age 25-29, total is the total population of 25-29 year olds in thousands estimated by Barro-Lee.

ECONOMIQUE : 

9 - GDP at purchaser's prices is the sum of gross value added by all resident producers in the economy plus any product taxes and minus any subsidies not included in the value of the products. It is calculated without making deductions for depreciation of fabricated assets or for depletion and degradation of natural resources. Data are in current U.S. dollars. Dollar figures for GDP are converted from domestic currencies using single year official exchange rates. For a few countries where the official exchange rate does not reflect the rate effectively applied to actual foreign exchange transactions, an alternative conversion factor is used.

10 - GDP per capita is gross domestic product divided by midyear population. GDP is the sum of gross value added by all resident producers in the economy plus any product taxes and minus any subsidies not included in the value of the products. It is calculated without making deductions for depreciation of fabricated assets or for depletion and degradation of natural resources. Data are in current U.S. dollars

11 - GNI per capita based on purchasing power parity (PPP). PPP GNI is gross national income (GNI) converted to international dollars using purchasing power parity rates. An international dollar has the same purchasing power over GNI as a U.S. dollar has in the United States. GNI is the sum of value added by all resident producers plus any product taxes (less subsidies) not included in the valuation of output plus net receipts of primary income (compensation of employees and property income) from abroad. Data are in current international dollars based on the 2011 ICP round.


J'extrais les series code afin de simplifier les analyses

In [ ]:
# Création DataFrame IndicateurSeries pour chercher les séries code. 
IndicateurSeries =Series[['Series Code','Indicator Name', 'Short definition', 'Long definition']]

# Indicateurs éducations 
resultats_education_sup = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('Total enrollment') & IndicateurSeries['Long definition'].str.contains('tertiary education') & IndicateurSeries['Long definition'].str.contains('ISCED')]
resultats_education_lycee = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('upper') & IndicateurSeries['Long definition'].str.contains('Total enrollment') & IndicateurSeries['Long definition'].str.contains('education')]

# Indicateurs techniques
resultats_technique_internet = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('Internet')]
resultats_technique_ordi = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('Personal') & IndicateurSeries['Long definition'].str.contains('computers')]

# Indicateurs démographique
resultats_démographique_totalPop = IndicateurSeries[IndicateurSeries['Series Code'].str.contains('SP.POP.TOTL') & IndicateurSeries['Long definition'].str.contains('Total population')]
resultats_démographique_1519 = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('Population') & IndicateurSeries['Long definition'].str.contains('thousands') & IndicateurSeries['Long definition'].str.contains('total') & IndicateurSeries['Long definition'].str.contains('15-19')]
resultats_démographique_2024 = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('Population') & IndicateurSeries['Long definition'].str.contains('thousands') & IndicateurSeries['Long definition'].str.contains('total') & IndicateurSeries['Long definition'].str.contains('20-24')]
resultats_démographique_2529 = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('Population') & IndicateurSeries['Long definition'].str.contains('thousands') & IndicateurSeries['Long definition'].str.contains('total') & IndicateurSeries['Long definition'].str.contains('25-29')]

# Indicateurs économiques
resultats_economique_PIB = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('GDP') & IndicateurSeries['Long definition'].str.contains('prices is the sum') & IndicateurSeries['Long definition'].str.contains('Data are in current U')]
resultats_economique_PIBHAB = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('GDP per capita') & IndicateurSeries['Long definition'].str.contains('midyear population') & IndicateurSeries['Long definition'].str.contains('Data are in current U')]
resultats_economique_RNB = IndicateurSeries[IndicateurSeries['Long definition'].str.contains('GNI per capita') & IndicateurSeries['Long definition'].str.contains('PPP GNI is gross national income')]

# Création DataFrame des indicateurs retenus
Indicateurs_total = [
    resultats_economique_PIB["Series Code"].iloc[0],
    resultats_economique_PIBHAB["Series Code"].iloc[0],
    resultats_economique_RNB["Series Code"].iloc[0],
    resultats_démographique_1519["Series Code"].iloc[0],
    resultats_démographique_2024["Series Code"].iloc[0],
    resultats_démographique_2529["Series Code"].iloc[0],
    resultats_démographique_totalPop["Series Code"].iloc[0],
    resultats_technique_internet["Series Code"].iloc[0],
    resultats_technique_ordi["Series Code"].iloc[0],
    resultats_education_sup["Series Code"].iloc[0],
    resultats_education_lycee["Series Code"].iloc[0]
]
Indicateurs_total


### Taux de remplissage

In [ ]:
plt.figure(figsize=(20,10))
Remplissage=Data.set_index('Country Name')
sns.heatmap(Remplissage.isna(), cbar=False)
plt.grid(color="red",linewidth=0.3)
plt.show() 

### Choix des Indicateurs et de la période 

In [ ]:
# Indicateurs sélectionnés
selectedData = Data[Data["Indicator Code"].isin(Indicateurs_total)]

# Période sélectionnée : 2000 - 2015
# colonnes à supprimer
colonnes_a_supprimer = [col for col in selectedData.columns if ('1970' <= col <= '1999') or (col >= '2016')]
selectedPeriod = selectedData.drop(columns=colonnes_a_supprimer[4:])


# Pays sélectionnés
FinalDf = selectedPeriod[selectedPeriod["Country Code"].isin(liste_PaysFinal["Country Code"])]


### Retrait des pays sans donnée

In [ ]:
# Df des lignes qui ont toutes des données manquantes tous les ans
noData = FinalDf.loc[(FinalDf['2000'].isnull()) &(FinalDf['2001'].isnull()) &(FinalDf['2002'].isnull()) &(FinalDf['2003'].isnull()) &(FinalDf['2004'].isnull()) &(FinalDf['2005'].isnull()) &(FinalDf['2006'].isnull()) &(FinalDf['2007'].isnull()) &(FinalDf['2008'].isnull()) &(FinalDf['2009'].isnull()) &(FinalDf['2010'].isnull()) & (FinalDf['2011'].isnull()) & (FinalDf['2012'].isnull()) & (FinalDf['2013'].isnull()) & (FinalDf['2014'].isnull()) & (FinalDf['2015'].isnull())]

pd.set_option('display.max_rows', None)
# On visualise les pays et les indicateurs concernés par le manque total de données
noData.loc[:,['Country Name','Country Code','Indicator Name']]
# liste des codes pays a supprimer
Pays_noData = noData['Country Code'].unique()
Pays_noData
# Suppression des pays sans donnée
col= FinalDf.columns
FinalDf = FinalDf[~FinalDf['Country Code'].isin(Pays_noData)][col]

In [ ]:
print(FinalDf.isnull().sum())

### Ajout de la région et du groupe de revenu

In [ ]:
Final = FinalDf.merge(Country[['Country Code', 'Region', 'Income Group']], on='Country Code', how='left')


Region = Final.pop("Region")
IncomeGroup = Final.pop("Income Group")

Final.insert(4, "Region", Region)
Final.insert(5, "Income Group", IncomeGroup)

Final.head()



### Tx de remplissage DF Final

In [ ]:
plt.figure(figsize=(15,7))
RemplissageFinalDF=Final.set_index('Country Name')
sns.heatmap(RemplissageFinalDF.iloc[:,5:].isna(), cbar=False)
plt.grid(color="white",linewidth=0.3)
plt.show() 

In [ ]:
# Taux de remplissage par indicateurs (sur le nombre de pays, i.e. sur 211)
CompteRemplissageParIndicateurs = RemplissageFinalDF.groupby(['Indicator Name']).count()
CompteRemplissageParIndicateurs =CompteRemplissageParIndicateurs.iloc[:,3:]
sns.heatmap(CompteRemplissageParIndicateurs)


### Graphique des indicateurs sur 2010

In [ ]:
#Graphiques statistiques sur les indicateurs  
plt.figure(figsize = (15, 20))


# ------------------------------------------------------------------------------------
# Indicateur démographique : SP.POP.TOTL
# ------------------------------------------------------------------------------------

df_eco=Final[Final['Indicator Code'] == 'SP.POP.TOTL']

plt.subplot(5,2,3)
sns.boxplot( x=df_eco['Indicator Code'], y=df_eco['2010'], width=0.5, color='SteelBlue')
plt.title('Indicateur économique')
plt.ylabel('RNB/habitant, PPA ($)')

plt.subplot(5,2,4)
sns.histplot(df_eco['2010'], kde = True, color='SteelBlue')
plt.title('Indicateur économique')

# ------------------------------------------------------------------------------------
# Indicateur économique : NY.GNP.PCAP.PP.CD
# ------------------------------------------------------------------------------------
df_eco=Final[Final['Indicator Code'] == 'NY.GNP.PCAP.PP.CD']

plt.subplot(5,2,3)
sns.boxplot( x=df_eco['Indicator Code'], y=df_eco['2010'], width=0.5, color='SteelBlue')
plt.title('Indicateur économique')
plt.ylabel('RNB/habitant, PPA ($)')

plt.subplot(5,2,4)
sns.histplot(df_eco['2010'], kde = True, color='SteelBlue')
plt.title('Indicateur économique')
plt.xlabel('NY.GNP.PCAP.PP.CD')

# ------------------------------------------------------------------------------------
# Indicateur éducatif lycée : SE.SEC.ENRR.UP
# ------------------------------------------------------------------------------------
df_edu_sec=Final[Final['Indicator Code'] == 'SE.SEC.ENRR.UP']

plt.subplot(5,2,5)
sns.boxplot( x=df_edu_sec['Indicator Code'], y=df_edu_sec['2010'], width=0.5, color='SteelBlue')
plt.title('Indicateur éducatif lycée')
plt.ylabel('Taux scolarisé lycée (%)')

plt.subplot(5,2,6)
sns.histplot(df_edu_sec['2010'], kde = True, color='SteelBlue')
plt.title('Indicateur éducatif lycée')
plt.xlabel('SE.SEC.ENRR.UP')

# ------------------------------------------------------------------------------------
# Indicateur éducatif lycée : SE.TER.ENRR
# ------------------------------------------------------------------------------------
df_edu_ter=Final[Final['Indicator Code'] == 'SE.TER.ENRR']

plt.subplot(5,2,7)
sns.boxplot( x=df_edu_ter['Indicator Code'], y=df_edu_ter['2010'], width=0.5, color='SteelBlue')
plt.title('Indicateur éducatif ens. sup.')
plt.ylabel('Taux scolarisé ens. sup. (%)')

plt.subplot(5,2,8)
sns.histplot(df_edu_ter['2010'], kde = True, color='SteelBlue')
plt.title('Indicateur éducatif ens. sup.')
plt.xlabel('SE.TER.ENRR')

# ------------------------------------------------------------------------------------
# Indicateur numérique : IT.NET.USER.P2
# ------------------------------------------------------------------------------------
df_num=Final[Final['Indicator Code'] == 'IT.NET.USER.P2']

plt.subplot(5,2,9)
sns.boxplot( x=df_num['Indicator Code'], y=df_num['2010'], width=0.5, color='SteelBlue')
plt.title('Indicateur numérique')
plt.ylabel('Taux d\'tilisateur d\'internet (%)')

plt.subplot(5,2,10)
sns.histplot(df_num['2010'], kde = True, color='SteelBlue')
plt.title('Indicateur numérique')
plt.xlabel('IT.NET.USER.P2')

plt.tight_layout()

### Indicateurs

In [ ]:
Stats_paysIndicateurs = Final.iloc[:,0:4]
Stats_paysIndicateurs.head()

In [ ]:
Stats_paysIndicateurs["mean"] = np.nanmean(Final.iloc[:,7:],axis=1)
Stats_paysIndicateurs["std"] = np.nanstd(Final.iloc[:,7:],axis=1)
Stats_paysIndicateurs["count"] = Final.iloc[:,7:].count(axis=1)
Stats_paysIndicateurs["Median"] = Final.iloc[:,7:].quantile(q=0.5,axis=1)
Stats_paysIndicateurs["Q1"] = Final.iloc[:,7:].quantile(q=0.25,axis=1)
Stats_paysIndicateurs["Q3"] = Final.iloc[:,7:].quantile(q=0.75,axis=1)
Stats_paysIndicateurs.head()

In [ ]:
Stats_paysIndicateurs.dropna().head()

In [ ]:
Final.head()

### Classement des pays

In [ ]:
ClassementMelt=Final.melt(id_vars=['Country Name','Indicator Code','Region','Income Group'],value_vars=['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015'], var_name='Year', value_name='Value')
ClassementPv=ClassementMelt.pivot_table(index=['Country Name','Region','Income Group'],columns='Indicator Code',values='Value')
ClassementPv = ClassementPv.fillna(0)
ClassementPv = ClassementPv.round().astype(int)
ClassementPv.head()

In [ ]:
ClassementPv.describe()

In [ ]:
#La population de chaque pays en % de la population du pays le plus peuplé
ClassementPv['POPULATION TOTALE']=ClassementPv['SP.POP.TOTL']/ClassementPv['SP.POP.TOTL'].max()*100

#Pourcentage des 15-19 ans sur la population totale ( l'indicateur est exprimé en milliers)
ClassementPv['POPULATION 15-19']=ClassementPv['BAR.POP.1519']*1000/ClassementPv['SP.POP.TOTL']*100

#Pourcentage des 20-24 ans sur la population totale ( l'indicateur est exprimé en milliers)
ClassementPv['POPULATION 20-24']=ClassementPv['BAR.POP.2529']*1000/ClassementPv['SP.POP.TOTL']*100

#Pourcentage des 25-29 ans sur la population totale ( l'indicateur est exprimé en milliers)
ClassementPv['POPULATION 25-29']=ClassementPv['BAR.POP.2024']*1000/ClassementPv['SP.POP.TOTL']*100

#PIB par habitant normalisé par le max des PIB par habitant
ClassementPv['PIB/HAB']=ClassementPv['NY.GDP.PCAP.CD']/ClassementPv['NY.GDP.PCAP.CD'].max()*100

#PIB normalisé par le max des PIB le plus élévé des pays retenus
ClassementPv['PIB']=ClassementPv['NY.GDP.MKTP.CD']/ClassementPv['NY.GDP.MKTP.CD'].max()*100

#Revenu national brut/habitant
ClassementPv['RNB/HAB']=ClassementPv['NY.GNP.PCAP.PP.CD']/ClassementPv['NY.GNP.PCAP.PP.CD'].max()*100

listeIndicateursNormalises=['POPULATION TOTALE','POPULATION 15-19','POPULATION 20-24','POPULATION 25-29','PIB/HAB','PIB','IT.NET.USER.P2','IT.CMP.PCMP.P2','SE.SEC.ENRR.UP','SE.TER.ENRR','RNB/HAB']
ClassementPourcent=ClassementPv.filter(items=listeIndicateursNormalises)
ClassementPourcent.head()

In [ ]:
ClassementPourcent['Score attractivite']=ClassementPourcent.mean(axis=1)
Classement=ClassementPourcent.sort_values('Score attractivite',ascending=False).head(20)
Classement

In [ ]:
plt.figure(figsize=(15, 6))
plt.bar(x=Classement.index.get_level_values(0), height=Classement['Score attractivite'])
plt.xlabel('Pays')
plt.ylabel('Score attractivite')
plt.title('Classement de 20 pays les plus attractifs')
plt.xticks(rotation=45)
plt.show()

### Classement des pays avec pondération

In [ ]:
#liste des poids des indicateurs
weight=[1,1,5,5,1,1,5,1,1,1,5]
#ajout d'une colonne contenant le score pondéré
Classement['Classement']=(
    weight[0]*Classement['POPULATION TOTALE']
    +weight[1]*Classement['POPULATION 15-19']
    +weight[2]*Classement['POPULATION 20-24']
    +weight[3]*Classement['POPULATION 25-29']
    +weight[4]*Classement['SE.SEC.ENRR.UP']
    +weight[5]*Classement['SE.TER.ENRR']
    +weight[6]*Classement['IT.NET.USER.P2']
    +weight[7]*Classement['IT.CMP.PCMP.P2']
    +weight[8]*Classement['PIB/HAB']
    +weight[9]*Classement['PIB']
    +weight[10]*Classement['RNB/HAB'])/sum(weight)

ClassementSup = Classement.sort_values('Classement', ascending=False).head(20)
#Représentation graphique du top 20 des pays avec une moyenne pondérée
ClassementSup['Pays'] = ClassementSup.index.get_level_values(0)

# Représentation graphique du top 20 des pays avec une moyenne pondérée
ax = ClassementSup.plot.bar(x='Pays', y='Classement', title="Classement des pays par attractivité")
ax.set_xlabel('Pays')  # Définition du label de l'axe des abscisses
plt.show()

In [ ]:
ClassementSup_Indicateurs = ['POPULATION TOTALE', 'POPULATION 15-19', 'POPULATION 20-24','POPULATION 25-29', 'PIB/HAB', 'PIB', 'IT.NET.USER.P2','IT.CMP.PCMP.P2', 'SE.SEC.ENRR.UP', 'SE.TER.ENRR', 'RNB/HAB']
Classement_Indicateurs = ClassementSup[ClassementSup_Indicateurs]
Classement_Indicateurs

### Répartition des régions

In [ ]:
ClassementSup = Classement.sort_values('Classement', ascending=False).head(20)
ClassementSup['Region'] = ClassementSup.index.get_level_values(1)

ax = ClassementSup['Region'].value_counts().plot.pie(title="Répartition des régions par attractivité", autopct='%1.1f%%')
ax.set_ylabel('')  # Supprimer le label de l'axe des ordonnées
plt.show()

In [ ]:
ClassementSup = Classement.sort_values('Classement', ascending=False).head(20)
ClassementSup['Revenu'] = ClassementSup.index.get_level_values(2)

ax = ClassementSup['Revenu'].value_counts().plot.pie(title="Répartition des régions par attractivité")
ax.set_ylabel('')  
plt.show()

### Tableau de corréalation entre indicateurs

In [ ]:
result_df = pd.DataFrame(index=Classement_Indicateurs.columns, columns=Classement_Indicateurs.columns)
pValue_df = pd.DataFrame(index=Classement_Indicateurs.columns, columns=Classement_Indicateurs.columns)
result_df = result_df.astype(float)
# Calculer les p-values pour chaque paire de colonnes
for i in range(len(Classement_Indicateurs.columns)):
    for j in range(len(Classement_Indicateurs.columns)):
        result_df.iloc[i, j], pValue_df.iloc[i, j] = pearsonr(Classement_Indicateurs.iloc[:, i], Classement_Indicateurs.iloc[:, j])

# Appliquer un style au DataFrame pour afficher la grille de couleur
fig = plt.figure(figsize=(10,8))
cm = sns.color_palette("Spectral",10)
result_df[pValue_df>0.05] = np.NaN
sns.heatmap(result_df, cmap=cm, vmin=0.5, vmax=1, xticklabels=Classement_Indicateurs.columns, yticklabels=Classement_Indicateurs.columns, annot=True, fmt=".2f")
plt.tight_layout()
fig.savefig('tchatcheur02.png')
plt.show()

In [ ]:
# Créer un DataFrame vide pour stocker les résultats
result_df = np.full((len(ClassementSup.columns),len(ClassementSup.columns)), np.nan)
pValue = np.full((len(ClassementSup.columns),len(ClassementSup.columns)), np.nan)
# Calculer les p-values pour chaque paire de colonnes
for i in range(len(ClassementSup.columns)):
    for j in range(len(ClassementSup.columns)):
        result_df[i, j], pValue[i,j] = pearsonr(ClassementSup.iloc[:, i], ClassementSup.iloc[:, j])

# Convertir les p-values en DataFrame pour l'affichage
result_df = result_df.astype(float)

# Appliquer un style au DataFrame pour afficher la grille de couleur
fig = plt.figure()
cm = sns.color_palette("Spectral",5)
sns.heatmap(result_df,cmap=cm, vmin=0.5, vmax=1, xticklabels= ClassementSup.columns, yticklabels= ClassementSup.columns, annot=True, fmt=".2f")
plt.tight_layout()
fig.savefig('tchatcheur.png')

### Box plot des indicateurs

In [ ]:
plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='POPULATION TOTALE')
plt.title('Boxplot de la population totale par pays')
plt.ylabel('Population totale')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='POPULATION 15-19')
plt.title('Boxplot de la population de 15 à 19')
plt.ylabel('Population 15 - 19')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='POPULATION 20-24')
plt.title('Boxplot de la population de 20 à 24')
plt.ylabel('Population 20 - 24')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='POPULATION 25-29')
plt.title('Boxplot de la population de 25 à 29')
plt.ylabel('Population 25 - 29')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='PIB/HAB')
plt.title('Boxplot du PIB')
plt.ylabel('PIB/HAB en %')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='PIB')
plt.title('Boxplot du PIB')
plt.ylabel('PIB en %')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='IT.NET.USER.P2')
plt.title('Boxplot d utilisateurs d internet')
plt.ylabel('Utilisateurs d internet')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='IT.CMP.PCMP.P2')
plt.title('Boxplot de personne ayant un ordinateur')
plt.ylabel('Untilisateurs d ordinateur')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='SE.SEC.ENRR.UP')
plt.title('Boxplot de la scolarisation au lycée')
plt.ylabel('Population au lycée')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='SE.TER.ENRR')
plt.title('Boxplot de la population à l université')
plt.ylabel('Population à l université')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

plt.figure(figsize=(10, 6))
ClassementSup.boxplot(column='RNB/HAB')
plt.title('Boxplot du RNB/Hab')
plt.ylabel('RNB/HAB en %')
plt.xlabel('Pays')
plt.xticks(rotation=45)  # Rotation des étiquettes de l'axe des x pour une meilleure lisibilité
plt.grid(False)
plt.show()

In [ ]:
# Créer une nouvelle figure avec une grille de sous-graphiques
plt.figure(figsize=(20, 18))

# Définir les titres des graphiques
titles = [
    'Boxplot de la population totale par pays',
    'Boxplot de la population de 15 à 19',
    'Boxplot de la population de 20 à 24',
    'Boxplot de la population de 25 à 29',
    'Boxplot du PIB/HAB',
    'Boxplot du PIB',
    'Boxplot d utilisateurs d internet',
    'Boxplot de personne ayant un ordinateur',
    'Boxplot de la scolarisation au lycée',
    'Boxplot de la population à l université',
    'Boxplot du RNB/Hab'
]

# Définir les noms des colonnes dans le dataframe
columns = [
    'POPULATION TOTALE',
    'POPULATION 15-19',
    'POPULATION 20-24',
    'POPULATION 25-29',
    'PIB/HAB',
    'PIB',
    'IT.NET.USER.P2',
    'IT.CMP.PCMP.P2',
    'SE.SEC.ENRR.UP',
    'SE.TER.ENRR',
    'RNB/HAB'
]

# Créer les subplots et tracer les boxplots
for i, column in enumerate(columns, 1):
    plt.subplot(4, 3, i)
    plt.boxplot(ClassementSup[column])
    plt.title(titles[i - 1])
    plt.ylabel('Valeurs')
    plt.xlabel('Pays')
    plt.xticks(rotation=45)
    plt.grid(False)

# Ajuster la disposition et l'espacement des sous-graphiques
plt.tight_layout()

# Afficher le graphique
plt.show()


In [ ]:
# Créer une nouvelle figure
plt.figure(figsize=(20, 18))

# Tracer tous les boxplots sur le même graphique
ClassementSup.boxplot(column=[
    'POPULATION TOTALE', 
    'POPULATION 15-19', 
    'POPULATION 20-24', 
    'POPULATION 25-29', 
    'PIB/HAB', 
    'PIB', 
    'IT.NET.USER.P2', 
    'IT.CMP.PCMP.P2', 
    'SE.SEC.ENRR.UP', 
    'SE.TER.ENRR', 
    'RNB/HAB'
], ax=plt.gca())

# Définir le titre, les étiquettes des axes et faire pivoter les étiquettes de l'axe x
plt.title('Boxplots des indicateurs par pays')
plt.ylabel('Valeurs')
plt.xlabel('Indicateurs')
plt.xticks(rotation=45)
plt.grid(False)

# Afficher le graphique
plt.show()

### Prédiction

In [ ]:
# Indicateurs sélectionnés
Prediction = Data[Data["Indicator Code"].isin(Indicateurs_total)]

# Période sélectionnée : 2020 - 2100
# colonnes à supprimer
colonnes_a_supprimer_prediction = [col for col in Prediction.columns if ('1970' <= col <= '2018')]
colonnes_a_supprimer_prediction.extend([col for col in Prediction.columns if ('1970' <= col <= '1973')])
selectedPeriod_Prediction = Prediction.drop(columns=colonnes_a_supprimer_prediction[4:])

# Pays sélectionnés
FinalDfPrediction = selectedPeriod_Prediction[selectedPeriod_Prediction["Country Code"].isin(liste_PaysFinal["Country Code"])]

In [ ]:
# Df des lignes qui ont toutes des données manquantes tous les ans
noDataPrediction = FinalDfPrediction.loc[(FinalDfPrediction['2020'].isnull()) &(FinalDfPrediction['2025'].isnull()) &(FinalDfPrediction['2030'].isnull()) &(FinalDfPrediction['2035'].isnull()) &(FinalDfPrediction['2040'].isnull()) &(FinalDfPrediction['2045'].isnull()) &(FinalDfPrediction['2050'].isnull()) &(FinalDfPrediction['2055'].isnull()) &(FinalDfPrediction['2060'].isnull()) &(FinalDfPrediction['2065'].isnull()) & (FinalDfPrediction['2070'].isnull()) & (FinalDfPrediction['2075'].isnull()) & (FinalDfPrediction['2080'].isnull()) & (FinalDfPrediction['2085'].isnull()) & (FinalDfPrediction['2090'].isnull()) & (FinalDfPrediction['2095'].isnull()) & (FinalDfPrediction['2100'].isnull())]
noDataPrediction
pd.set_option('display.max_rows', None)
# On visualise les pays et les indicateurs concernés par le manque total de données
noDataPrediction.loc[:,['Country Name','Country Code','Indicator Name']]
# liste des codes pays a supprimer
Pays_noData_Prediction = noDataPrediction['Country Code'].unique()
Pays_noData_Prediction
# Suppression des pays sans donnée
col= FinalDfPrediction.columns
FinalDfPrediction = FinalDfPrediction[~FinalDfPrediction['Country Code'].isin(Pays_noData_Prediction)][col]
print(FinalDfPrediction.isnull().sum())